# Binance Hawkes Calibration

Calibrate Hawkes models on Binance BTCUSDT trades preprocessed via `scripts/preprocess_binance.py`.

## 1. Load processed sequences

In [ ]:

from pathlib import Path
import numpy as np
import torch
from torch.utils.data import DataLoader

import neural_hawkes as nh

processed_dir = Path('data/runs/events')
combined_times = np.load(processed_dir / 'BTCUSDT-2025-09-21-combined-times.npy')
combined_marks = np.load(processed_dir / 'BTCUSDT-2025-09-21-combined-marks.npy')

sequence = nh.EventSequence(times=combined_times, types=combined_marks)
sequences = [sequence]


## 2. Classical Hawkes (tick)

In [ ]:

try:
    from tick.hawkes import HawkesExpKern
    learner = HawkesExpKern(decays=2.0)
    learner.fit([sequence.times])
    print('mu =', learner.baseline)
    print('adjacency =', learner.adjacency)
except Exception as exc:
    print('tick not available or fit failed:', exc)


## 3. Neural Hawkes training

In [ ]:

dataset = nh.EventSequenceDataset(sequences, window_size=64, stride=32)
train_set, val_set, test_set = nh.split_dataset(dataset, (0.7, 0.15, 0.15))
collate = nh.collate_windows
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, collate_fn=collate)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, collate_fn=collate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = nh.NeuralHawkesModel(num_types=2, embed_dim=32, hidden_dim=64, backbone='gru').to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, 4):
    train_metrics = nh.train_one_epoch(model, train_loader, optim, device, delta_weight=1.0)
    val_metrics = nh.evaluate(model, val_loader, device, delta_weight=1.0)
    print(f"Epoch {epoch}: train {train_metrics['loss']:.4f}, val {val_metrics['loss']:.4f}")


## 4. Diagnostics

In [ ]:

pred_deltas, true_deltas, mask = nh.collect_predictions(model, test_loader, device)
nh.time_rescaling_diagnostics(pred_deltas, true_deltas, mask)
